In [1]:
%%bash
git clone https://github.com/cailinw/rl-project.git

Cloning into 'rl-project'...


In [8]:
%%bash
cd rl-project/
git pull
cd ..

Updating 5ab4e7b..facd7e2
Fast-forward
 imagecoco/map_gpt2.py     | 20 ++++++++++----------
 imagecoco/rewarder.py     |  9 +++------
 imagecoco/test_modules.py | 27 ++++++++++++++++++---------
 imagecoco/utils.py        |  5 +++--
 4 files changed, 34 insertions(+), 27 deletions(-)


From https://github.com/cailinw/rl-project
   5ab4e7b..facd7e2  main       -> origin/main


In [2]:
%%bash
pip install transformers

  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=9c310075780799e5c9125496c14a9b94d09c9d19cc9eb345180d93fd80f983bd
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
import sys
sys.path.append("rl-project/imagecoco/")

import torch

# from dataloader import Dataloader
from rewarder import Rewarder, RewardModel
from generator import Generator

import pickle

In [12]:
class Test:
    def __init__(
        self, seq_length, batch_size, hidden_state_size, embed_dim, mlp_hidden_size
    ):
        self.seq_length = seq_length
        self.batch_size = batch_size
        self.vocab_size = 4348
        self.hidden_state_size = hidden_state_size
        self.embed_dim = embed_dim
        self.mlp_hidden_size = mlp_hidden_size
        self.learning_rate = 0.01

    def test_reward_model(self):
        hidden_state_size = 64

        model = RewardModel(
            hidden_state_size=hidden_state_size,
            mlp_hidden_size=self.mlp_hidden_size,
            embed_size=self.embed_dim,
            vocab_size=self.vocab_size,
        )

        x = torch.randn((self.batch_size, hidden_state_size))
        a = torch.randint(self.vocab_size, (self.batch_size,))

        result = model(x, a)

        assert result.shape[0] == self.batch_size
        assert result.shape[1] == 1

    def test_rewarder_rewards_to_go(self):
        rewarder = Rewarder(
            self.seq_length,
            self.batch_size // 2,
            self.batch_size // 2,
            self.vocab_size,
            self.hidden_state_size,
            self.embed_dim,
            self.mlp_hidden_size,
            self.learning_rate,
        )
        trajectories = torch.randint(
            self.vocab_size, (self.batch_size, self.seq_length)
        )
        str_map = pickle.load(open("rl-project/imagecoco/save/str_map.pkl", "rb"))
        generator = Generator(self.seq_length, str_map)
        rewarder.rewards_to_go(trajectories, generator, roll_num=4)

    def test_dataloader(self):
        train_data = COCOImageCaptionsDataset("save/train_data.pkl")
        train_dataloader = DataLoader(train_data, batch_size=512, shuffle=True)
        for batch_idx, (truth, m_in, mask) in enumerate(train_dataloader):
            print("batch_idx: ", batch_idx)
            print(truth.shape, m_in.shape, mask.shape)
            break

    def runtests(self):
        self.test_reward_model()
        self.test_rewarder_rewards_to_go()
        # self.test_rewarder_train_step()
        # self.test_dataloader()

In [13]:
test = Test(32, 64, 512, 100, 128)

In [14]:
test.test_reward_model()

FileNotFoundError: ignored

['', '/env/python', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/root/.ipython', 'rl-project/imagecoco/']
